In [215]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bertopic import BERTopic

from nltk import word_tokenize
import os.path as op
import re 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [216]:
# import the preprocess doc

df = pd.read_csv('./Data_csv/data_preprocessed.csv')

In [217]:
df.columns

Index(['doc_id', 'Name of the document', 'Institution', 'URL', 'Authors',
       'Affiliates', 'Sector', 'Country', 'Date', 'Keywords',
       'Exclusion criteria', 'Status', 'Label', 'MapAIE (ours)', 'Jobin',
       'Fjeld', 'Tidjon', 'Hagendorff', 'Floridi', 'Zeng (LAIP)',
       'Attard-Frost', 'EP', 'Algorithm watch', 'CE', 'Winfield',
       'EthicalML GitHub', 'all sources', 'Checked by', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'text', 'langue', 'text_processed',
       'tfidf', 'categorie Institution', 'theme'],
      dtype='object')

In [218]:
df.count()

doc_id                   457
Name of the document     457
Institution              457
URL                      457
Authors                   33
Affiliates                29
Sector                    42
Country                  446
Date                      44
Keywords                   6
Exclusion criteria         0
Status                   457
Label                    394
MapAIE (ours)            457
Jobin                     59
Fjeld                     21
Tidjon                    24
Hagendorff                12
Floridi                    4
Zeng (LAIP)               42
Attard-Frost              28
EP                         8
Algorithm watch          103
CE                       377
Winfield                  15
EthicalML GitHub          13
all sources              457
Checked by               457
Unnamed: 27                2
Unnamed: 28                0
Unnamed: 29                2
text                     457
langue                   457
text_processed           457
tfidf         

In [219]:
documents = df['text_processed'].to_list()

In [220]:
vectorizer = CountVectorizer(stop_words='english')
Bow = vectorizer.fit_transform(documents)

# Calculer la fréquence totale de chaque mot
frequency = Bow.toarray().sum(axis=0)

# Obtenir les 20 mots les plus fréquents
top_indices = np.argsort(frequency)[-20:]  # Indices des 20 mots les plus fréquents
vocabulary = np.array(vectorizer.get_feature_names_out())  # Le vocabulaire
top_words = vocabulary[top_indices] 

stopwords_to_remove = set(top_words)

dates = set([str(i) for i in range(1990, 2021)])
useless_words = set(('may','also','see','system', 'must', 'et', 'al'))

html_stopwords = set({'maxwidth', 'px', 'maxheight', 'img', 'src', 'https', 'com', 'www', 'http', 'jpg', 'png', 'gif', 'jpeg', 'pdf', 'html'})
css_stopwords = set({'grid','body', ':root', '\n', 'padding',',button','margintop', 'px', 'margin', 'border', 'width', 'height', 'color', 'font', 'size', 'background', 'position', 'left', 'right', 'top', 'bottom', 'display', 'flex', 'align', 'justify', 'content', 'center', 'float', 'clear', 'overflow', 'hidden', 'zindex', 'cursor', 'pointer', 'hover', 'active', 'focus', 'transition', 'transform', 'rotate', 'scale', 'translate', 'opacity', 'box', 'shadow', 'outline', 'none', 'block', 'inline', 'inlineblock', 'relative', 'absolute', 'fixed', 'static', 'sticky', 'visible', 'hidden'})



stopwords_to_remove = stopwords_to_remove.union(dates)
stopwords_to_remove = stopwords_to_remove.union(html_stopwords)
stopwords_to_remove = stopwords_to_remove.union(css_stopwords)
stopwords_to_remove = stopwords_to_remove.union(useless_words)

# remove all possible dates day and month from the documents (such as 22 march...)

def remove_months_days(doc, dates):
    for date in dates:
        doc = doc.replace(date, "")
    return doc

dates = set([str(i) for i in range(1990, 2025)])
dates = dates.union(set(['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']))
dates = dates.union(set(['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']))


documents = [remove_months_days(doc, dates) for doc in documents]





def remove_frequent_words(doc, stopwords):
    words = doc.split()
    cleaned_words = [word for word in words if word not in stopwords]
    return " ".join(cleaned_words)


documents = [remove_frequent_words(doc, stopwords_to_remove) for doc in documents]



# BERTopic (sans n-gram)

In [221]:
import hdbscan

hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=1, cluster_selection_epsilon=0.1)
topic_model = BERTopic(hdbscan_model=hdbscan_model, language="english", nr_topics=10)

In [222]:
n_topics = 0
while n_topics < 8:
    topics, probs = topic_model.fit_transform(documents)
    n_topics = len(set(topics))

In [223]:
df['topic'] = topics
df['probs'] = probs

In [224]:
# Afficher les topics les plus fréquents
print(topic_model.get_topic_info())



   Topic  Count                                          Name  \
0     -1    115         -1_national_policy_based_technologies   
1      0     36           0_protection_law_personal_standards   
2      1     33           1_government_uk_strategy_innovation   
3      2     40  2_algorithmic_algorithms_bias_decisionmaking   
4      3     77              3_europe_energy_legal_commission   
5      4     65        4_international_policy_work_government   
6      5     40                       5_ethics_ethical_ts_wha   
7      6     51               6_machine_ethical_ethics_health   

                                      Representation  \
0  [national, policy, based, technologies, report...   
1  [protection, law, personal, standards, technol...   
2  [government, uk, strategy, innovation, support...   
3  [algorithmic, algorithms, bias, decisionmaking...   
4  [europe, energy, legal, commission, technologi...   
5  [international, policy, work, government, coop...   
6  [ethics, ethical, t

In [225]:
print(topic_model.get_topic(-1)) 

[('national', 0.011850001856304576), ('policy', 0.010559448391368765), ('based', 0.009971611186680534), ('technologies', 0.00973394877093243), ('report', 0.009152244247335677), ('recognition', 0.0086519741431865), ('law', 0.008358261926282138), ('machine', 0.008338212431854291), ('strategy', 0.008237399970469858), ('training', 0.008213251869832794)]


In [226]:
# Visualisation des topics
topic_model.visualize_topics()


In [227]:

# Visualisation des hiérarchies entre les topics
topic_model.visualize_hierarchy()

In [228]:

# Visualisation des relations entre les topics
topic_model.visualize_barchart(n_words=10, height=400, width=400)

In [229]:
topic_model.visualize_heatmap(n_clusters=7)

ValueError: Make sure to set `n_clusters` lower than the total number of unique topics.

In [ ]:
topic_model.get_representative_docs(0)

["strasbourg 25 t-pd 09rev consultative committee convention protection individuals regard automatic processing personal convention 108 report protection challenges possible remedies directorate general rule law 2 report alessandro mantelero associate professor private law polytechnic university turin department management production engineering document expression author ’ personal viewpoint contents part – state art ................................ ................................ ................................ .............. 3 i.1 introduction ................................ ................................ ................................ ................................ .. 3 i.2 devel opment ................................ ................................ ................................ ............................ 4 i.3 perspective adopted ................................ ................................ ................................ ........... 5 i.4 existing framework p

In [ ]:

# Reclassification des outliers
topic_model.reduce_outliers(documents, topics=topics)

# Obtention des nouveaux topics
new_topics, new_probs = topic_model.transform(documents)

AttributeError: No prediction data was generated

In [ ]:
print(topic_model.get_topic_info())

# Visualisation
topic_model.visualize_topics()

In [ ]:
df[df["topic"] == -1]

,doc_id,Name of the document,Institution,URL,Authors,Affiliates,Sector,Country,Date,Keywords,...,Unnamed: 28,Unnamed: 29,text,langue,text_processed,tfidf,categorie Institution,theme,topic,probs
0,3,Intel Recommends Public Policy Principles for ...,Intel,https://community.intel.com/t5/Blogs/Intel/Pol...,Naveen Rao,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,\n\n\n\tIntel Recommends Public Policy Princip...,en,intel recommends public policy principles arti...,[0.01722362 0. 0. ... 0. ...,Entreprises technologiques et multinationales,Privacy,-1,0.0
3,13,The Toronto Declaration: Protecting the right ...,Access Now,https://www.torontodeclaration.org/declaration...,"Anna Bacciarelli, Joe Westby, Fanny Hidvegi, E...","Access Now, AI Now Institute at New York Unive...",civil society,UK,2018-05-16,NaN,...,NaN,NaN,\n\n\n\n\nThe Toronto Declaration\n\n\n\n\n\n\...,en,toronto declaration skip main content toronto ...,[0. 0. 0. ... 0. 0. 0.],ONG et initiatives de droits humains,Justice and fairness,-1,0.0
24,39,AI and Human Rights,All Tech is Human,https://alltechishuman.org/ai-human-rights-report,NaN,NaN,NaN,USA,NaN,NaN,...,NaN,NaN,\n\n\n\n\n\n\nAI and Human Rights: Building a ...,en,ai human rights building tech future aligned p...,[0. 0. 0. ... 0. 0. 0.],Associations professionnelles et groupes de ré...,Sustainable,-1,0.0
28,46,Safety First for Automated Driving – Proposed ...,Aptiv,https://www.heise.de/downloads/18/2/7/0/8/1/7/...,NaN,NaN,NaN,Germany,NaN,NaN,...,NaN,NaN,2019 SAFETY FIRST FOR AUTOMATED DRIVING \nI \...,en,2019 safety first automated driving ii iii aut...,[0. 0.00153116 0. ... 0. ...,Entreprises technologiques et multinationales,Non-maleficence,-1,0.0
31,49,It’s Time to Do Something: Mitigating the Nega...,Association for Computing Machinery - Future o...,https://perma.cc/K22T-5DFU,NaN,NaN,NaN,USA,NaN,NaN,...,NaN,NaN,\n\nPerma | It’s Time to Do Something: Mitigat...,en,perma ’ time something mitigating negative imp...,[0. 0. 0. ... 0. 0. 0.],Associations professionnelles et groupes de ré...,Transparency,-1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,702,Recommendations on Updating the National Artif...,University of Stanford,https://hai.stanford.edu/sites/default/files/2...,NaN,NaN,NaN,USA,NaN,NaN,...,NaN,NaN,Recommendations on Updating the National Art...,en,recommendations updating national artificial i...,[0. 0. 0. ... 0. 0. 0.],Instituts de recherche et universités,Non-maleficence,-1,0.0
440,706,AI UX: 7 principles of designing Good AI Products,UX Studio Team,https://uxstudioteam.com/ux-blog/ai-ux/,Dávid Pásztor,NaN,civil society,Hungary,2018-04-17,NaN,...,NaN,NaN,AI UX: 7 Principles of Designing Good AI Produ...,en,ai ux 7 principles designing good ai products ...,[0. 0. 0. ... 0. 0. 0.],Associations professionnelles et groupes de ré...,Trust,-1,0.0
441,710,Global competition and convergence of AI Law,Vrije Universiteit Brussel,https://osf.io/preprints/socarxiv/j36ke/,NaN,NaN,NaN,Belgium,NaN,NaN,...,NaN,NaN,\n\n\n\nOSF\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,en,osf full functionality site necessary enable j...,[0. 0. 0. ... 0. 0. 0.],Instituts de recherche et universités,Transparency,-1,0.0
445,715,Guidance for Regulation of Artificial Intellig...,White House (US),https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN,NaN,USA,NaN,NaN,...,NaN,NaN,EXECUTIVE OFFICE OF THE PRESIDENT OFFICE OF M...,en,executive office president office management b...,[0. 0. 0. ... 0. 0. 0.],Gouvernements et organismes publics nationaux,Non-maleficence,-1,0.0


In [ ]:
df.head()


,doc_id,Name of the document,Institution,URL,Authors,Affiliates,Sector,Country,Date,Keywords,...,Unnamed: 28,Unnamed: 29,text,langue,text_processed,tfidf,categorie Institution,theme,topic,probs
0,3,Intel Recommends Public Policy Principles for ...,Intel,https://community.intel.com/t5/Blogs/Intel/Pol...,Naveen Rao,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,\n\n\n\tIntel Recommends Public Policy Princip...,en,intel recommends public policy principles arti...,[0.01722362 0. 0. ... 0. ...,Entreprises technologiques et multinationales,Privacy,-1,0.0
1,9,Artificial intelligence in Healthcare,Academy of Medical Royal Colleges,https://www.aomrc.org.uk/wp-content/uploads/20...,NaN,NaN,NaN,UK,NaN,NaN,...,NaN,NaN,Artificial Intelligence in HealthcareJanuary /...,en,artificial intelligence healthcarejanuary 2019...,[0. 0.00872152 0. ... 0. ...,Instituts de recherche et universités,Trust,0,1.0
2,12,Human rights in the age of Artificial Intellig...,Access Now,https://www.accessnow.org/cms/assets/uploads/2...,.,.,NaN,NaN,NaN,NaN,...,NaN,NaN,HUMAN RIGHTS IN THE AGE OF ARTIFICIAL INTELLI...,en,human rights age artificial intelligence repor...,[0. 0.0087539 0. ... 0. 0...,ONG et initiatives de droits humains,Justice and fairness,6,1.0
3,13,The Toronto Declaration: Protecting the right ...,Access Now,https://www.torontodeclaration.org/declaration...,"Anna Bacciarelli, Joe Westby, Fanny Hidvegi, E...","Access Now, AI Now Institute at New York Unive...",civil society,UK,2018-05-16,NaN,...,NaN,NaN,\n\n\n\n\nThe Toronto Declaration\n\n\n\n\n\n\...,en,toronto declaration skip main content toronto ...,[0. 0. 0. ... 0. 0. 0.],ONG et initiatives de droits humains,Justice and fairness,-1,0.0
4,14,Europe’s approach to artificial intelligence: ...,AccessNow,https://www.accessnow.org/wp-content/uploads/2...,NaN,NaN,NaN,USA,2020-12,NaN,...,NaN,NaN,accessnow.org EUROPE’S APPROACH TO ARTIFICIAL ...,en,accessnow.org europe ’ approach artificial int...,[0. 0.0035214 0. ... 0. 0...,ONG et initiatives de droits humains,Transparency,7,1.0


# BERTopic avec N-gram

In [230]:


n_gram_range = (2, 3)

hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=1, cluster_selection_epsilon=0.1)
topic_model = BERTopic(hdbscan_model=hdbscan_model, language="english", nr_topics=10, n_gram_range=n_gram_range)

In [231]:
n_topics = 0
while n_topics < 8:
    topics, probs = topic_model.fit_transform(documents)
    n_topics = len(set(topics))

In [232]:
df['topic'] = topics
df['probs'] = probs

In [233]:
# Afficher les topics les plus fréquents
print(topic_model.get_topic_info())



   Topic  Count                                               Name  \
0     -1    139  -1_facial recognition_decision making_member s...   
1      0     35  0_ethics commission_decision making_related te...   
2      1     33  1_industrial strategy_strategy white_industria...   
3      2     32  2_climate change_united nations_discussion pap...   
4      3     91  3_member states_industrial revolution_third in...   
5      4     31  4_algorithmic decisionmaking_centre ethics_bia...   
6      5     67  5_united states_index report_autonomous intell...   
7      6     29  6_computers learn_promise computers learn_prom...   

                                      Representation  \
0  [facial recognition, decision making, member s...   
1  [ethics commission, decision making, related t...   
2  [industrial strategy, strategy white, industri...   
3  [climate change, united nations, discussion pa...   
4  [member states, industrial revolution, third i...   
5  [algorithmic decisionmaking, c

In [234]:
print(topic_model.get_topic(-1)) 

[('facial recognition', 0.002628323241856115), ('decision making', 0.002132439657376694), ('member states', 0.0017168516264077832), ('council europe', 0.00166487076085066), ('covid 19', 0.0013953907143980596), ('private sector', 0.001280593061468446), ('social media', 0.001197988508807949), ('law enforcement', 0.001191830092961012), ('freedom expression', 0.0011648321361954108), ('united states', 0.0011479970307570636)]


In [235]:
# Visualisation des topics
topic_model.visualize_topics()


In [236]:

# Visualisation des hiérarchies entre les topics
topic_model.visualize_hierarchy()

In [237]:

# Visualisation des relations entre les topics
topic_model.visualize_barchart(n_words=10, height=400, width=400)

In [238]:
topic_model.visualize_heatmap(n_clusters=7)

ValueError: Make sure to set `n_clusters` lower than the total number of unique topics.